In [ ]:
!apt-get update
!apt-get install -y tesseract-ocr libtesseract-dev

!pip install -U torch torchvision

!pip install -U cython
!pip install -U 'git+https://github.com/facebookresearch/fvcore.git'
!pip install -U 'git+https://github.com/cocodataset/cocoapi.git#subdirectory=PythonAPI'

!git clone https://github.com/facebookresearch/detectron2.git
%cd detectron2
!pip install -e .
%cd ..

!pip install -U matplotlib numpy opencv-python-headless pandas tqdm tabulate gradio streamlit huggingface_hub pytesseract

import torch
import torchvision
import detectron2
#import encapsu_view
import gradio as gr
import cv2
import matplotlib.pyplot as plt
import pytesseract

print("PyTorch:", torch.__version__)
print("Detectron2:", detectron2.__version__)

In [ ]:
!pip install git+https://github.com/MaksymTymkovych/alginate-capsules-analysis


In [ ]:
import zipfile, json, io, os, shutil, numpy as np
import cv2
import matplotlib.pyplot as plt

from encapsu_view.analysis.detection import process_image_with_capsules, capsule_detector, inner_detector
from encapsu_view.analysis.scale import scale_detector
from encapsu_view.visualization.hierarchy import visualize_tree, plot_capsule_hierarchy

from encapsu_view.analysis.scale import create_color_range_from_bgr, detect_scale_ruller, detect_lines_from_mask, find_longest_horizontal_line, extract_text_from_ruler_region


def load_image_cv(file_path):
    img = cv2.imread(file_path, cv2.IMREAD_UNCHANGED)
    if img is None:
        raise ValueError(f"Failed to read image: {file_path}")

    if len(img.shape) == 2:
        if img.dtype == np.uint16:
            img = (img / 256).astype(np.uint8)
        img = cv2.cvtColor(img, cv2.COLOR_GRAY2RGB)
        return img

    if img.shape[2] == 3:
        return cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    if img.shape[2] == 4:
        return cv2.cvtColor(img, cv2.COLOR_BGRA2RGB)

    raise ValueError(f"Unsupported shape: {img.shape}")

def run_capsule_analysis(image_path):
    img_rgb = load_image_cv(image_path)

    img_bgr = cv2.cvtColor(img_rgb, cv2.COLOR_RGB2BGR)

    detected_data = process_image_with_capsules(img_bgr, capsule_detector, inner_detector, scale_detector)
    processed_data = detected_data.convert_to_processed_data(ellipse_method="direct")

    raw_vis = cv2.cvtColor(detected_data.visualize_detection(img_bgr), cv2.COLOR_BGR2RGB)
    processed_vis = cv2.cvtColor(processed_data.visualize_detection(img_bgr), cv2.COLOR_BGR2RGB)

    print("\n--- Capsule Tree ---")
    visualize_tree(processed_data)

    json_data = processed_data.to_dict()

    plot_capsule_hierarchy(processed_data)

    return raw_vis, processed_vis, json_data


def process_batch(zip_file_path):
    temp_in = "temp_batch"
    temp_out = "temp_batch_out"
    for p in [temp_in, temp_out]:
        if os.path.exists(p):
            shutil.rmtree(p)
        os.makedirs(p)

    with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
        zip_ref.extractall(temp_in)

    results = {}
    output_images = []

    for fname in os.listdir(temp_in):
        if not fname.lower().endswith((".png", ".jpg", ".jpeg", ".bmp", ".tiff")):
            continue
        path = os.path.join(temp_in, fname)
        img = cv2.imread(path)
        if img is None:
            continue

        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        edges = cv2.Canny(gray, 80, 180)

        out_img = cv2.cvtColor(edges, cv2.COLOR_GRAY2RGB)
        out_path = os.path.join(temp_out, fname)
        cv2.imwrite(out_path, cv2.cvtColor(out_img, cv2.COLOR_RGB2BGR))
        output_images.append(out_path)

        results[fname] = {"shape": list(img.shape), "edges_detected": int(np.sum(edges > 0))}

    zip_out_path = "processed_out.zip"
    with zipfile.ZipFile(zip_out_path, 'w') as zipf:
        for f in os.listdir(temp_out):
            zipf.write(os.path.join(temp_out, f), arcname=f)

    return output_images, results, zip_out_path


if __name__ == "__main__":
    image_path = "./Snap-21062.tif"  
    raw_vis, processed_vis, json_data = run_capsule_analysis(image_path)
    #print("\nProcessed JSON:")
    #print(json.dumps(json_data, indent=2))



In [ ]:
import gradio as gr
import zipfile, json, io, os, shutil, numpy as np
import cv2
import matplotlib.pyplot as plt

from encapsu_view.analysis.detection import process_image_with_capsules, capsule_detector, inner_detector
from encapsu_view.analysis.scale import scale_detector
from encapsu_view.visualization.hierarchy import visualize_tree, plot_capsule_hierarchy

def load_image_cv(file_obj):
    if isinstance(file_obj, str):
        img = cv2.imread(file_obj, cv2.IMREAD_UNCHANGED)
    else:
        file_bytes = np.frombuffer(file_obj.read(), np.uint8)
        img = cv2.imdecode(file_bytes, cv2.IMREAD_UNCHANGED)

    if img is None:
        raise ValueError("Failed to read image")

    if len(img.shape) == 2:
        if img.dtype == np.uint16:
            img = (img / 256).astype(np.uint8)
        img = cv2.cvtColor(img, cv2.COLOR_GRAY2RGB)
        return img

    if img.shape[2] == 3:
        return cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    if img.shape[2] == 4:
        return cv2.cvtColor(img, cv2.COLOR_BGRA2RGB)

    raise ValueError(f"Unsupported shape: {img.shape}")


def run_capsule_analysis(image_file):
    img_rgb = load_image_cv(image_file)
    img_bgr = cv2.cvtColor(img_rgb, cv2.COLOR_RGB2BGR)

    detected_data = process_image_with_capsules(img_bgr, capsule_detector, inner_detector, scale_detector)
    processed_data = detected_data.convert_to_processed_data(ellipse_method="direct")

    raw_vis = cv2.cvtColor(detected_data.visualize_detection(img_bgr), cv2.COLOR_BGR2RGB)
    processed_vis = cv2.cvtColor(processed_data.visualize_detection(img_bgr), cv2.COLOR_BGR2RGB)

    tree_buffer = io.StringIO()
    print("Tree Structure:", file=tree_buffer)
    visualize_tree(processed_data)
    tree_text = tree_buffer.getvalue()
    
    json_data = processed_data.to_dict()

    fig = plt.figure(figsize=(6, 4))
    buf = io.BytesIO()
    plt.savefig(buf, format="png")
    plt.close(fig)
    buf.seek(0)
    hierarchy_plot = buf.getvalue()

    return raw_vis, processed_vis, json_data, tree_text, processed_vis


def process_batch(zip_file):
    temp_in = "/content/temp_batch"
    temp_out = "/content/temp_batch_out"
    for p in [temp_in, temp_out]:
        if os.path.exists(p):
            shutil.rmtree(p)
        os.makedirs(p)

    with zipfile.ZipFile(zip_file.name, 'r') as zip_ref:
        zip_ref.extractall(temp_in)

    results = {}
    output_images = []

    for fname in os.listdir(temp_in):
        if not fname.lower().endswith((".png", ".jpg", ".jpeg", ".bmp", ".tiff")):
            continue
        path = os.path.join(temp_in, fname)
        img = cv2.imread(path)
        if img is None:
            continue

        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        edges = cv2.Canny(gray, 80, 180)

        out_img = cv2.cvtColor(edges, cv2.COLOR_GRAY2RGB)
        out_path = os.path.join(temp_out, fname)
        cv2.imwrite(out_path, cv2.cvtColor(out_img, cv2.COLOR_RGB2BGR))
        output_images.append(out_path)

        results[fname] = {"shape": list(img.shape), "edges_detected": int(np.sum(edges > 0))}

    zip_out_path = "/content/processed_out.zip"
    with zipfile.ZipFile(zip_out_path, 'w') as zipf:
        for f in os.listdir(temp_out):
            zipf.write(os.path.join(temp_out, f), arcname=f)

    return output_images, json.dumps(results, indent=2), zip_out_path


with gr.Blocks(title="Alginate Capsule Analysis") as demo:
    gr.Markdown("Alginate Capsule Analysis")

    with gr.Tabs():
        with gr.Tab("Single File"):
            img_in = gr.File(label="Upload Image", file_types=[".jpeg",".png",".jpg",".tiff",".tif"])
            raw_img = gr.Image(label="Raw Detection")
            proc_img = gr.Image(label="Processed Detection")
            json_out = gr.JSON(label="Processed JSON")
            tree_out = gr.Textbox(label="Capsule Tree", lines=20)
            plot_out = gr.Image(label="Hierarchy Plot")
            btn = gr.Button("Run Analysis")
            btn.click(run_capsule_analysis, inputs=[img_in], outputs=[raw_img, proc_img, json_out, tree_out, plot_out])

        with gr.Tab("Batch ZIP"):
            zip_in = gr.File(label="Upload ZIP with images", file_types=[".zip"])
            gallery_out = gr.Gallery(label="Processed Images", columns=4)
            json_batch = gr.JSON(label="Batch JSON")
            zip_out = gr.File(label="Download Processed ZIP")
            btn2 = gr.Button("Process ZIP (OpenCV)")
            btn2.click(process_batch, inputs=zip_in, outputs=[gallery_out, json_batch, zip_out])

demo.launch(share=True)